In [91]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import json

LANGUAGE = "en"

In [92]:
# Loading tags 
with open('../../data/pragueeu/pragueeu_tags_export.json') as f:
    data_tags = json.load(f)

tags = []
print("Number of tags available:", len(data_tags))
for tag in data_tags:
    row = {}
    row["id"] = tag["id"]
    row["sys_name"] = tag["systemId"]
    for lang in tag["names"]:
        if lang["lang"] == LANGUAGE:
            row["name"] = lang["name"]
    tags.append(row)

df_tags = pd.DataFrame(tags)
df_tags.set_index("id", drop=True, inplace=True)
df_tags.head()

NUmber of tags available: 234


,sys_name,name
id,,
280,PAMATKYAARCHITEKTURA,Monuments & Architecture
3,dum,house
15,"fontany,kasny",fountain
5,hrad,castle/chateau
11,hrbitov,cemetery


In [93]:
# open destination file
with open('../../data/pragueeu/pragueeu_objects_export.json') as f:
    data = json.load(f)

rows = []

print("Number of places available:", len(data))
for place in data:
    row = {}
    row["id"] = place["id"]
    
    if "address" in place and place["address"]:
        s = place["address"]["street"]
        d = place["address"]["district"]
        t = place["address"]["town"] 
        z = place["address"]["zipCode"]  
        if s and d and t and z:
            row["address"] = s+ ", " + d + ", " + t + ", " + z
    if "gps" in place and place["gps"]:
        gps = place["gps"]
        y = gps["latitude"]
        x = gps["longitude"]
        if x and y:
            row["geometry"] = Point(x,y)
    for lang in place["texts"]:
        if lang["lang"] == LANGUAGE:
            row["name"] = lang["name"]
            row["text"] = lang["shortDescription"]

    for tag_id in place["tagIds"]:
        if tag_id in df_tags.index:
            tag_name = df_tags.loc[tag_id,"name"]
            row[tag_name] = True
    rows.append(row)

Number of places available: 466


In [94]:
rows[:1]

[{'id': 752,
  'address': 'Plzeňská 29, Smíchov, Praha 5, 150 00',
  'geometry': <shapely.geometry.point.Point at 0x7f14c48f4e20>,
  'name': 'EA Hotel Praga 1885',
  'text': 'EA Hotel Praga 1885 is a 4-star hotel in the broader city centre, which interior was inspired by the style at the end of the 19th century. It features 42 spacious rooms and suites, which are equipped with historicist furniture. Breakfast is served here every day. The historical core of metropolis can be reached within a 20-minute drive. ',
  'pets allowed': True,
  'hotel ****': True,
  'in the broader city centre': True}]

In [95]:
df = gpd.GeoDataFrame(rows, geometry="geometry", crs=4326)

In [115]:
df.columns;

In [110]:
df.geometry = df.geometry.to_crs(5514);

In [111]:
df.head(1)

,id,address,geometry,name,text,pets allowed,hotel ****,in the broader city centre,parking,hotel ***,...,hotel **,multifunctional space,Art Nouveau,observatory/planetarium,concert hall,modern architecture,botanical garden,botel,Classicism & Empire,walks
0,752,"Plzeňská 29, Smíchov, Praha 5, 150 00",POINT (-744808.176 -1044450.855),EA Hotel Praga 1885,EA Hotel Praga 1885 is a 4-star hotel in the b...,True,True,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
jdf = df.to_json(na="drop");

In [113]:
jdf;

In [114]:
# Using a JSON string
path = "../../data/pragueeu/tourism_dest.json"
with open(path, 'w') as outfile:
    outfile.write(jdf)